In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
os.chdir('/home/hrai/codes/PoseAdaptor')
from lib_import import *
from my_utils import *

# 주요 디렉토리 경로
user = getpass.getuser()
alphapose_root = '/home/{}/codes/AlphaPose'.format(user)
motionbert_root = '/home/{}/codes/MotionBERT'.format(user)
kookmin_root = '/home/{}/Datasets/HAAI/국민대데이터/data'.format(user)
ap_kookmin_result_root = alphapose_root + "/examples/kookmin_result_5actions"

assert os.path.isdir(alphapose_root), "AlphaPose root directory is not exist"
assert os.path.isdir(motionbert_root), "MotionBERT root directory is not exist"
assert os.path.isdir(kookmin_root), "Kookmin root directory is not exist"
assert os.path.isdir(ap_kookmin_result_root), "AlphaPose Kookmin result directory is not exist"

alphapose_root, motionbert_root, kookmin_root, ap_kookmin_result_root

('/home/hrai/codes/AlphaPose',
 '/home/hrai/codes/MotionBERT',
 '/home/hrai/Datasets/HAAI/국민대데이터/data',
 '/home/hrai/codes/AlphaPose/examples/kookmin_result_5actions')

In [2]:
os.chdir(motionbert_root)
from lib.utils.tools import * # get_config
from lib.utils.learning import * # load_backbone
from lib.data.datareader_kookmin import DataReaderKOOKMIN
from lib.data.dataset_motion_3d import MotionDataset3D

def parse_args(input_args=[]):
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", type=str, default="configs/pose3d/MB_ft_h36m.yaml", help="Path to the config file.")
    parser.add_argument('-c', '--checkpoint', default='checkpoint', type=str, metavar='PATH', help='checkpoint directory')
    parser.add_argument('-p', '--pretrained', default='checkpoint', type=str, metavar='PATH', help='pretrained checkpoint directory')
    parser.add_argument('-r', '--resume', default='', type=str, metavar='FILENAME', help='checkpoint to resume (file name)')
    parser.add_argument('-e', '--evaluate', default='checkpoint/pose3d/MB_ft_h36m/best_epoch.bin', type=str, metavar='FILENAME', help='checkpoint to evaluate (file name)')
    parser.add_argument('-ms', '--selection', default='latest_epoch.bin', type=str, metavar='FILENAME', help='checkpoint to finetune (file name)')
    parser.add_argument('-sd', '--seed', default=0, type=int, help='random seed')
    parser.add_argument('-g', '--gpu', default='0', type=str, help='GPU id')
    opts = parser.parse_args(input_args)
    return opts

In [5]:
model_list = ['MB_ft_h36m', 'FT_MB_release_kookmin_total_with_kookmin_setting_s0258_test']
target_list = ['squat_test', 'squat_test_front']
input_type_list = ['frame', 'video']

for model_name in model_list:
    for target in target_list:
        for input_type in input_type_list:
            print('model_name: {}, target: {}, input_type: {}'.format(model_name, target, input_type))
            input_args = ['--evaluate', 'checkpoint/pose3d/{}/best_epoch.bin'.format(model_name)]
            opts = parse_args(input_args)
            args = get_config(opts.config)

            model_backbone = load_backbone(args)
            model_backbone = nn.DataParallel(model_backbone)
            model_backbone = model_backbone.cuda()

            chk_filename = opts.evaluate
            print('Loading checkpoint', chk_filename)
            checkpoint = torch.load(chk_filename, map_location=lambda storage, loc: storage)
            model_backbone.load_state_dict(checkpoint['model_pos'], strict=True)
            model_pos = model_backbone
            
            # custom ap result
            ap_root = '/home/hrai/codes/AlphaPose/examples'
            ap_result_path = os.path.join(ap_root, target)
            video_path = os.path.join(ap_root, target + '.mp4')
            W, H, video_num_frames, fps = get_video_info(video_path)
            pose_2d_list = get_ap_pose_2d(video_path, ap_result_path)
            
            if input_type == 'frame':
                output = []
                for frame in range(243):
                    input_data = pose_2d_list[frame].copy().reshape(-1, 17, 3) # 1 frame
                    input_data = normalize_input(input_data, W, H) # normalize
                    input_data = torch.Tensor(input_data).unsqueeze(0) # to batch tensor
                    #print(input_data.shape)
                    output.append(get_inference_from_motionbert(model_pos, input_data, args, W, H)[0][0])
                output = np.array(output).reshape(243, 17, 3)
            elif input_type == 'video':
                input_data = pose_2d_list[:243].copy() # 243 frames
                input_data = normalize_input(input_data, W, H) # normalize
                input_data = torch.Tensor(input_data).unsqueeze(0) # to batch tensor
                output = get_inference_from_motionbert(model_pos, input_data, args, W, H)[0]
            #print(output.shape)
            save_h36m_pose_video(output, './{}_{}_{}.mp4'.format(model_name, target, input_type), pose_type='3d', refine_tilt=False)

model_name: MB_ft_h36m, target: squat_test, input_type: frame
Loading checkpoint checkpoint/pose3d/MB_ft_h36m/best_epoch.bin


100%|██████████| 243/243 [00:11<00:00, 20.44it/s]


model_name: MB_ft_h36m, target: squat_test, input_type: video
Loading checkpoint checkpoint/pose3d/MB_ft_h36m/best_epoch.bin


100%|██████████| 243/243 [00:12<00:00, 19.31it/s]


model_name: MB_ft_h36m, target: squat_test_front, input_type: frame
Loading checkpoint checkpoint/pose3d/MB_ft_h36m/best_epoch.bin


...

100%|██████████| 243/243 [00:12<00:00, 20.22it/s]


model_name: FT_MB_release_kookmin_total_with_kookmin_setting_s0258_test, target: squat_test_front, input_type: frame
Loading checkpoint checkpoint/pose3d/FT_MB_release_kookmin_total_with_kookmin_setting_s0258_test/best_epoch.bin


100%|██████████| 243/243 [00:12<00:00, 20.01it/s]


model_name: FT_MB_release_kookmin_total_with_kookmin_setting_s0258_test, target: squat_test_front, input_type: video
Loading checkpoint checkpoint/pose3d/FT_MB_release_kookmin_total_with_kookmin_setting_s0258_test/best_epoch.bin


100%|██████████| 243/243 [00:12<00:00, 19.95it/s]
